# Computational Methods in Simulation (CMIS) Week 3
This notebook is intended to help students make their hand-in in this week of CMIS.


Hand-ins must be submitted as a maximum 3 page pdf file in Absalon and must be made using the ACM TOG overleaf latex template

https://www.overleaf.com/latex/templates/association-for-computing-machinery-acm-large-2-column-format-template/qwcgpbmkkvpq

Notebooks must be uploaded as well so teachers have code solutions accessible if needed.

This week is based on Slides 6

* Explain how a mesh is represented and create a function that can generate a 2D traingle mesh for a beam (large rectangle) shape
* Apply a third-party meshing tool such as tetgen, wildmeshing or similar to a simple 2D case.
* Apply a 3D meshing tool such as TetGen or wildmeshing or similar to a 3D surface mesh such as 'armadillo.obj'
* Identify two or more quality measures from the Shewchuck paper that you believe will be good measures for a linear tetrahedral mesh. Explain pros and cons of your choosen measures (Hint: We provide functions for computing inscribed/circumscriped spheres as well as signed volume).
* Implement the chosen quality measures and create a tool for plotting quality historgrams.
* Compute your chosen quality measures for the 3D volume mesh you generate and display histograms of them. Discuss the histograms, what do they tell you about the mesh you generated?
* Implement the marching triangle method (as described in Slides 6) and identify "bad" or "wrong" triangles created by the method. Discuss the cause of these cases and if time permit develop an improved version of the method (Hint: reuse your signed distance field code from Week 2).  


## Detailed Expected Learning Objectives in this Week

* Computational Meshing (Slides 6)
    * Explain common mesh quality measures.
    * Implement a simple computational mesh generator.
    * Generate computational quality meshes using 3rd party software such as DistMesh, TetGen or similar.
    * Experimentally evaluate whether a given computational mesh is a quality mesh.

## Mesh Representations

In this work we use the face-indexed array representation for storing a unstructured mesh. That is a triangle mesh. One may find a short description of mesh descriptions from the libigl library

https://libigl.github.io/libigl-python-bindings/tutorials/

For our 3D meshing examples we may make use of TetGen as bundled in libigl

https://libigl.github.io/libigl-python-bindings/igl_docs/#tetrahedralize

and we make use of the the wildmeshing library for both 3D and 2D meshing

https://wildmeshing.github.io/wildmeshing-notebook/

We also provide two of our own primitive 2D meshing methods.

In [2]:
%matplotlib widget

In [3]:
import igl as igl
import meshplot as mp
import numpy as np
import wildmeshing as wm
import matplotlib.pyplot as plt

For starters we will show an example of how to directly generate a beam mesh.

The idea is to make the beam on a regular back ground grid. In a first sweep over grid nodes we create the vertices of the beam mesh. In a second sweep over the cells of the regular grid we will create triangles. 

To avoid having any bias from mesh connectivity we choose to flip the diagonals of the grid cells. This way we get a very homogenous looking triangle mesh with similar shaped triangles that are close to being equilateral.

In [3]:
def make_beam_mesh(width, height, shape):
    x0 = -width/2.0
    y0 = -height/2.0
    I  = shape[0]
    J  = shape[1]
    dx = width/float(I)
    dy = height/float(J)
    V = np.zeros(((I+1)*(J+1),2),dtype=np.float64)
    for j in range(J+1):
        for i in range(I+1):
            k = i + j*(I+1)
            V[k,0] = x0 + i*dx
            V[k,1] = y0 + j*dy
    T = np.zeros((2*I*J,3),dtype=np.int32)
    for j in range(J):
        for i in range(I):
            k00 = (i  ) + (j  )*(I+1)
            k01 = (i+1) + (j  )*(I+1)
            k10 = (i  ) + (j+1)*(I+1)
            k11 = (i+1) + (j+1)*(I+1)
            e = 2*(i + j*I)
            if (i+j)%2:
                T[e,  :] = (k00,k01,k11)
                T[e+1,:] = (k00,k11,k10)
            else:
                T[e,  :] = (k10,k00,k01)
                T[e+1,:] = (k10,k01,k11)                    
    return V, T

V, T = make_beam_mesh(2.0,1.0,(12,6))

p = mp.plot(V, T, return_plot=True, shading={"wireframe": True})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Next we will show an example using the wildmeshing library to generate a triangle mesh from a polygon boundary. 

In [4]:
# Documentation is here https://wildmeshing.github.io/wildmeshing-notebook/
def make_circle_mesh(radius, segments):
    K = segments
    X = np.array([ radius*np.cos(2*np.pi*k/K) for k in range(K)])
    Y = np.array([ radius*np.sin(2*np.pi*k/K) for k in range(K)])
    P = np.zeros((K,2))
    P[:,0] = X
    P[:,1] = Y
    L = np.array([(k,(k+1)%K) for k in range(K)],dtype=np.int32)
    
    V, T, _, _ = wm.triangulate_data(P, L, cut_outside=True)
    return (V, T)

V, T = make_circle_mesh(0.75, 36)
p = mp.plot(V, T, return_plot=True, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Quality Measures

We have identified 4 quality measures for linear tetrahedral meshes. We implement simple functions that given one element can compute its quality measure.

For computing quality measures it is helpful to be able to compute the signed volume of a tetrahedron, test if is left oriented, circumscribed and inscriped spheres.

In [70]:
def compute_signed_volume(pi, pj, pk, pm):
    return np.dot((pm - pi), np.cross((pj - pi), (pk - pi))) / 6.0


def is_left_orientation(pi, pj, pk, pm):
    return compute_signed_volume(pi, pj, pk, pm) < 0.0


def compute_circumscribed_sphere(p0, p1, p2, p3):
    A = p0 - p3
    B = p1 - p3
    C = p2 - p3

    M00 = np.dot(A, A)
    M01 = np.dot(A, B)
    M02 = np.dot(A, C)
    M10 = np.dot(B, A)
    M11 = np.dot(B, B)
    M12 = np.dot(B, C)
    M20 = np.dot(C, A)
    M21 = np.dot(C, B)
    M22 = np.dot(C, C)

    M = np.array([[M00, M01, M02], [M10, M11, M12], [M20, M21, M22]], dtype=np.float64)
    q = 0.5 * np.array([M00, M11, M22], dtype=np.float64)
    
    u = np.linalg.lstsq(M, q, rcond=None)[0]

    u0 = u[0]
    u1 = u[1]
    u2 = u[2]
    u3 = 1.0 - u0 - u1 - u2

    radius = np.linalg.norm(u0 * A + u1 * B + u2 * C)
    center = u0 * p0 + u1 * p1 + u2 * p2 + u3 * p3

    return center, radius


def compute_inscribed_sphere(pi, pj, pk, pm):
    nm = np.cross((pj - pi), (pk - pi))
    ni = np.cross((pm - pj), (pk - pj))
    nj = np.cross((pm - pk), (pi - pk))
    nk = np.cross((pm - pi), (pj - pi))
    Lnm = np.linalg.norm(nm)
    Lni = np.linalg.norm(ni)
    Lnj = np.linalg.norm(nj)
    Lnk = np.linalg.norm(nk)
    nm = nm / Lnm
    ni = ni / Lni
    nj = nj / Lnj
    nk = nk / Lnk

    wm = np.dot(nm, pi)
    wi = np.dot(ni, pj)
    wj = np.dot(nj, pk)
    wk = np.dot(nk, pi)

    M00 = nm[0]
    M01 = nm[1]
    M02 = nm[2]
    M03 = -1.0

    M10 = ni[0]
    M11 = ni[1]
    M12 = ni[2]
    M13 = -1.0

    M20 = nj[0]
    M21 = nj[1]
    M22 = nj[2]
    M23 = -1.0

    M30 = nk[0]
    M31 = nk[1]
    M32 = nk[2]
    M33 = -1.0

    A = np.array([[M00, M01, M02, M03], [M10, M11, M12, M13], [M20, M21, M22, M23], [M30, M31, M32, M33]],
                 dtype=np.float64)
    b = np.array([wm, wi, wj, wk], dtype=np.float64)

    x = np.linalg.lstsq(A, b, rcond=None)[0]

    center = np.array( [x[0], x[1], x[2]], dtype=np.float64)
    radius = x[3]

    return center, radius

Now we may implement our quality measures using these basic geometry functions

In [76]:
def compute_quality_nonsmooth_rms(pi, pj, pk, pm):
    V = compute_signed_volume(pi, pj, pk, pm)
    
    Lij = np.linalg.norm(pi - pj) 
    Lik = np.linalg.norm(pi - pk)
    Lim = np.linalg.norm(pi - pm)
    Ljk = np.linalg.norm(pj - pk)
    Ljm = np.linalg.norm(pj - pm)
    Lkm = np.linalg.norm(pk - pm)
    l_sum = Lij + Lik + Lim + Ljk + Ljm + Lkm
    l_rms = np.sqrt(l_sum / 6.0) ** 3
    
    const = 6 * np.sqrt(2)
    return const * (V / l_rms)

     
def compute_quality_nonsmooth_max(pi, pj, pk, pm):
    V = compute_signed_volume(pi, pj, pk, pm)
    
    Lij = np.linalg.norm(pi - pj) 
    Lik = np.linalg.norm(pi - pk)
    Lim = np.linalg.norm(pi - pm)
    Ljk = np.linalg.norm(pj - pk)
    Ljm = np.linalg.norm(pj - pm)
    Lkm = np.linalg.norm(pk - pm)
    L_max = max(Lij, Lik, Lim, Ljk, Ljm, Lkm) ** 3
    
    const = 6 * np.sqrt(2)
    return const * (V / L_max)


def compute_quality_knupp(pi, pj, pk, pm):
    V = compute_signed_volume(pi, pj, pk, pm)
    
    Lij = np.linalg.norm(pi - pj) ** 2 
    Lik = np.linalg.norm(pi - pk) ** 2
    Lim = np.linalg.norm(pi - pm) ** 2
    Ljk = np.linalg.norm(pj - pk) ** 2
    Ljm = np.linalg.norm(pj - pm) ** 2
    Lkm = np.linalg.norm(pk - pm) ** 2
    
    l_sum = Lij + Lik + Lim + Ljk + Ljm + Lkm
    l_rms = np.sqrt(l_sum / 6.0) 
    
    Ai = np.linalg.norm(np.cross((pj - pk), (pk - pm))) / 2 
    Aj = np.linalg.norm(np.cross((pi - pk), (pk - pm))) / 2 
    Ak = np.linalg.norm(np.cross((pi - pj), (pi - pm))) / 2 
    Am = np.linalg.norm(np.cross((pj - pk), (pi - pk))) / 2 
    A_sum = Ai ** 2 + Aj ** 2 + Ak ** 2 + Am ** 2 
    A_rms = np.sqrt(A_sum / 4)
    
    const = (3 * np.sqrt(6)) / 2.0
    return const * (V / (l_rms * A_rms + 0.000001))
     

def compute_quality_radius_ratio(pi, pj, pk, pm):
    r_out = compute_circumscribed_sphere(pi, pj, pk, pm)[1]
    r_in = compute_inscribed_sphere(pi, pj, pk, pm)[1]
    
    return 3 * r_in / r_out


Finally we can create a function that iterates over all tetrahedra in a mesh, compute quality measures for each tetrahedron and make histograms of the measures

In [77]:
def quality_histogram(vertices, tets, quality):
    histogram = np.zeros((tets.shape[0], 1), dtype = np.float64)
    
    X = vertices[:, 0]
    Y = vertices[:, 1]
    Z = vertices[:, 2]
    
    for n in range(tets.shape[0]):
        i = tets[n, 0]
        j = tets[n, 1]
        k = tets[n, 2]
        m = tets[n, 3]
        
        pi = np.array([X[i], Y[i], Z[i]])
        pj = np.array([X[j], Y[j], Z[j]])
        pk = np.array([X[k], Y[k], Z[k]])
        pm = np.array([X[m], Y[m], Z[m]])
        
        if is_left_orientation(pi, pj, pk, pm):
            pj, pi = pi, pj
            
        histogram[n] = quality(pi, pj, pk, pm)
        
    return histogram


In [92]:
def plot_quality_histograms(vertices, tets):
    rms = quality_histogram(vertices, tets, compute_quality_nonsmooth_rms)
    maxx = quality_histogram(vertices, tets, compute_quality_nonsmooth_max)
    knupp = quality_histogram(vertices, tets, compute_quality_knupp)
    rr = quality_histogram(vertices, tets, compute_quality_radius_ratio)
    
    fig, axs = plt.subplots(2, 2, figsize=(7, 7))
    axs[0, 0].hist(rms, bins=40, color='blue')
    axs[0, 0].set_title('(1)')
    axs[0, 1].hist(maxx, bins=40, color='red')
    axs[0, 1].set_title('(2)')
    axs[1, 0].hist(knupp,bins=40, color='green')
    axs[1, 0].set_title('(3)')
    axs[1, 1].hist(rr, bins=40,color='orange')
    axs[1, 1].set_title('(4)')
    plt.show()
    

# Reading Surface Mesh from File

We want to get some test geometry for making 3D tetrahedral meshes. We will use a popular computer graphics figure for our testing.

In [59]:
v, f = igl.read_triangle_mesh("armadillo.obj")
mp.plot(v, f, v[:, 0])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0002725…

# Create Volumetric Tetrahedral Mesh from Surface Mesh

We can now invoke a meshing routine to get a volume mesh and then visualize it.

In [12]:
#V, T, _ = igl.tetrahedralize(v, f, "pq1.414a0.01")

In [79]:
tetra = wm.Tetrahedralizer(stop_quality=500)
tetra.load_mesh("armadillo.obj")
tetra.tetrahedralize()
V, T = tetra.get_tet_mesh()
mp.plot(V, T)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0003275…

# Histograms of Quality Measures from 3D Volume Mesh

We now have all pieces in place to make histograms. We can use the histograms to get an idea of the quality of the mesh we have created.

In [93]:
plot_quality_histograms(V, T)

C:\Users\DKMEQ\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Just for fun we will play with a few more meshes to see how they behave

In [19]:
tetra = wm.Tetrahedralizer(stop_quality=500)
tetra.load_mesh("left_brain.stl")
tetra.tetrahedralize()
V, T = tetra.get_tet_mesh()
mp.plot(V, T)
plot_quality_histograms(V, T)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-33.78362…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
tetra = wm.Tetrahedralizer(stop_quality=500)
tetra.load_mesh("right_brain.stl")
tetra.tetrahedralize()
V, T = tetra.get_tet_mesh()
mp.plot(V, T)
plot_quality_histograms(V, T)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(34.877559…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Marching Triangles

We will reuse some of our code from week 2. Particular we need the function that can compute a signed distance field. This is because we want to apply the marching triangle method on a signed distance field.

In [23]:
def vec2(x,y):
    return np.array([x,y], dtype=np.float64)


def compute_signed_distance(poly, q):
    
    def is_left(p0, p1, p2):
        return (p1[0] - p0[0]) * (p2[1] - p0[1]) - (p2[0] - p0[0]) * (p1[1] - p0[1])

    N = poly.shape[0]
    d = np.inf

    # Compute distance to closest surface point
    for i in range(N-1):
        a = poly[i, :]    # Point on the surface of polygon p
        b = poly[i+1, :]  # Adjacent point on the surface of polygon p
        
        ba = b - a     # Vector from a to b
        bal = np.linalg.norm(ba) # Length of ba vector
        if bal == 0:
            continue
        ban = ba / bal # Normal of the vector from a to b
        qa = q - a     # The vector from the query point to a
        alpha = np.dot(qa, ban) / bal
        if alpha < 0:
            da = np.linalg.norm(q-a)
            if da < d:
                d = da
        elif alpha > 1:
            db = np.linalg.norm(q-b)
            if db < d:
                d = db
        else:
            banp = np.array([-ban[1], ban[0]])
            dp = abs(np.dot(qa, banp))
            if dp < d:
                d = dp
    # Compute winding number to set the sign
    wn = 0
    for i in range(N-1):
        a = poly[i, :]    # Point on the surface of polygon p
        b = poly[i+1, :]  # Adjacent point on the surface of polygon p
        if a[1] <= q[1]:
            if b[1] > q[1]:
                if is_left(a, b, q) > 0:
                    wn = wn + 1
        else:
            if b[1] <= q[1]:
                if is_left(a, b, q) < 0:
                    wn = wn -1
    if wn < -0.5:
        d = -d
    elif wn > 0.5:
        d = -d
    return d

The marching triangles method creates a regular background grid and then evaluates the signed distance field value at all grid points. Hereafer each cell is visited iteratively. The cell is cut into two triangles and each triangle is investigated. The triangle vertex order are shuffled such that all interior vertices comes first. 

We may now analyse the triangle case by assigning a case number to the triangle from a binary mask created from its inside-outside pattern of the vertices. The case number identifies a stencil for how to "cut" the triangle and generate triangles that covers the interior of the distance field.

In [37]:

def is_inside(poly, p):
    d = compute_signed_distance(poly, p)
    
    return d <= 0


def point(poly, p_in, p_out):
    d1 = compute_signed_distance(poly, p_in)
    d2 = compute_signed_distance(poly, p_out)
    dist = d2 - d1
    coef = -d1 / dist

    p = p_in + (p_out - p_in) * coef
    
    return p

    
def two_outside(poly, p1, p2, p3):
    n1 = point(poly, p2, p1)
    n3 = point(poly, p2, p3)
    
    return np.array([n1, p2, n3])
  
    
def one_outside(poly, p1, p2, p3):
    intersection_1 = point(poly, p2, p1)
    intersection_2 = point(poly, p3, p1)
    
    return intersection_1, intersection_2
  

def marching_triangles(poly, I, J):
    V, T = make_beam_mesh(3.0, 3.0, (I, J))
    X = V[:, 0]
    Y = V[:, 1]
    
    ind = 0
    v = np.empty((0,2),dtype=np.float64)
    t = np.empty((0,3),dtype=np.float64)
    
    for triangle in T:
        p1 = np.array([X[triangle[0]], Y[triangle[0]]])
        phi_1 = is_inside(poly, p1)
            
        p2 = np.array([X[triangle[1]], Y[triangle[1]]])
        phi_2 = is_inside(poly, p2)
                
        p3 = np.array([X[triangle[2]], Y[triangle[2]]])
        phi_3 = is_inside(poly, p3)
            
        if phi_1 and phi_2 and phi_3:
            v = np.append(v, [p1, p2, p3], axis=0)
            t = np.append(t, [[ind, ind + 1, ind + 2]], axis=0)
            
            ind = ind + 3
            
            
        elif phi_1 and phi_2:
            int1, int2 = one_outside(poly, p3, p1, p2)
            
            v = np.append(v, [p1, int1, int2], axis=0)
            v = np.append(v, [p1, p2, int2], axis=0)
            
            t = np.append(t, [[ind, ind + 1, ind + 2]], axis=0)
            ind = ind + 3
            
            t = np.append(t, [[ind, ind + 1, ind + 2]], axis=0)
            ind = ind + 3
            
            
        elif phi_1 and phi_3:
            int1, int2 = one_outside(poly, p2, p3, p1)
            
            v = np.append(v, [p3, int1, int2], axis=0)
            v = np.append(v, [p3, p1, int2], axis=0)
            
            t = np.append(t, [[ind, ind + 1, ind + 2]], axis=0)
            ind = ind + 3
            
            t = np.append(t, [[ind, ind + 1, ind + 2]], axis=0)
            ind = ind + 3  
            
            
        elif phi_2 and phi_3:
            int1, int2 = one_outside(poly, p1, p2, p3)
            
            v = np.append(v, [p3, int1, int2], axis=0)
            v = np.append(v, [p2, p3, int1], axis=0)
            
            t = np.append(t, [[ind, ind + 1, ind + 2]], axis=0)
            ind = ind + 3
            
            t = np.append(t, [[ind, ind + 1, ind + 2]], axis=0)
            ind = ind + 3
            
            
        elif phi_1:
            ps =  two_outside(poly, p2, p1, p3)
            v = np.append(v, ps, axis=0)
            t = np.append(t, [[ind, ind + 1, ind + 2]], axis=0)
            
            ind = ind + 3
            
            
        elif phi_2:
            ps = two_outside(poly, p1, p2, p3)
            v = np.append(v, ps, axis=0)
            t = np.append(t, [[ind, ind + 1, ind + 2]], axis=0)
            
            ind = ind + 3
            
        elif phi_3:
            ps = two_outside(poly, p1, p3, p2)
            v = np.append(v, ps, axis=0)
            t = np.append(t, [[ind, ind + 1, ind + 2]], axis=0)
            
            ind = ind + 3
            
        else: 
            continue
        
        
    return v, t


We may now apply the marching triangle to a test problem

In [40]:
poly = np.zeros((9,2),dtype=np.float64)
poly[0,:] = vec2(-1,-1)
poly[1,:] = vec2(-0.5,-1)
poly[2,:] = vec2(-0.5,0.5)
poly[3,:] = vec2(0.5,0.5)
poly[4,:] = vec2(0.5,-1)
poly[5,:] = vec2( 1,-1)
poly[6,:] = vec2( 1, 1)
poly[7,:] = vec2(-1, 1)
poly[8,:] = vec2(-1,-1)

V, T = make_beam_mesh(3.0, 3.0, (16, 16))

v, t = marching_triangles(poly, 16, 16)

fig = plt.figure()
#plt.plot(poly[:,0],poly[:,1])
#plt.triplot(V[:,0], V[:,1], T)
plt.triplot(v[:,0], v[:,1], t, c='red')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We may observe several issues with the current implementation. Particular at corners we can see sampling side effects. The method can be fooled to believe the whole area of a triangle is part of the inside if all vertices are inside. Likewise it can be fooled to think a triangle is completely outside if none of the vertices are interior. For instance the in the case of the polygon overlapping only one edge of the triangle.

Besides these sampling artefacts we may also note that the current implementation generates redundant vertices. When a triangle is cut by the zero-level set then the intersection points that is computed are not re-used when dealing with neighboring triangles that share the same intersection points.

That is it folks!